In [ ]:
# Load the Drive helper and mount
from google.colab import drive
import os

drive.mount('/content/drive')
path = "placeholder-path"
os.chdir(path)

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torch.optim import Adam
import numpy as np

## Load Data

In [1]:
folder_name = 'PokemonData'
model_data = datasets.ImageFolder(folder_name)

NameError: ignored

## Data Preprocessing

In [ ]:
#Define transformations for the training set and test set

train_transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

test_transformations = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

In [ ]:
# Randomly split into train, validation, and test sets

total_count = len(model_data)
train_count = int(0.7 * total_count)
valid_count = int(0.2 * total_count)
test_count = total_count - train_count - valid_count
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    model_data, (train_count, valid_count, test_count)
)

In [ ]:
# Apply transformations accordingly

train_dataset.dataset.transform = train_transformations
valid_dataset.dataset.transform = test_transformations
test_dataset.dataset.transform = test_transformations

In [ ]:
# Load training & validation sets for training

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


## Load Model

In [ ]:
# Define model architecture
def get_model():
    model = nn.Sequential(
          nn.Conv2d(3, 64, 3, stride=1, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Conv2d(64, 128, 3, stride=1, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout(p=0.3),
          nn.BatchNorm2d(128),
          nn.Conv2d(128, 128, 3, stride=1, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Conv2d(128, 256, 3, stride=1, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.BatchNorm2d(256),
          nn.Conv2d(256, 256, 3, stride=1, padding='same'),
          nn.ReLU(),
          nn.MaxPool2d(2),
          nn.Dropout(p=0.3),
          nn.Flatten(),
          nn.Linear(256*8*8, 1024),
          nn.ReLU(),
          nn.Dropout(p=0.3),
          nn.Linear(1024, 512),
          nn.ReLU(),
          nn.Linear(512, 150),
          # nn.Softmax(dim=1) # exclude Softmax layer as Torch applies it during prediction
        )

    return model

# Function to adjust learning rate over epoch
def adjust_learning_rate(epoch):

    lr = 0.001

    if epoch > 180:
        lr = lr / 1000000
    elif epoch > 150:
        lr = lr / 100000
    elif epoch > 120:
        lr = lr / 10000
    elif epoch > 90:
        lr = lr / 1000
    elif epoch > 60:
        lr = lr / 100
    elif epoch > 30:
        lr = lr / 10

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

# Function to get validation accuracy
def validation():
    model.eval()
    valid_acc = 0.0
    for i, (images, labels) in enumerate(validation_loader):
      
        if cuda_avail:
              images = images.cuda()
              labels = labels.cuda()

        #Predict classes using images from the test set
        outputs = model(images)
        _,prediction = torch.max(outputs.data, 1)
        valid_acc += torch.sum(prediction == labels.data)
        


    #Compute the average acc and loss over all 10000 test images
    valid_acc = valid_acc / valid_count

    return valid_acc
  
# Function to save models that are better than before
def save_models(epoch):
    torch.save(model.state_dict(), "pytorch_model_{}.model".format(epoch))
    print("Checkpoint saved")

## Define training parameters

In [ ]:
# Check if gpu support is available
cuda_avail = torch.cuda.is_available()

# Create model, optimizer and loss function
model = get_model()

#if cuda is available, move the model to the GPU
if cuda_avail:
    model.cuda()

#Define the optimizer and loss function
optimizer = Adam(model.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

## Training

In [ ]:
best_acc = 0.0
for epoch in range(30):
      print("epoch: {}".format(epoch))
      model.train()
      train_acc = 0.0
      train_loss = 0.0
      for i, (images, labels) in enumerate(train_loader):
          #Move images and labels to gpu if available
          if cuda_avail:
              images = images.cuda()
              labels = labels.cuda()

          #Clear all accumulated gradients
          optimizer.zero_grad()
          #Predict classes using images from the test set
          outputs = model(images)
          #Compute the loss based on the predictions and actual labels
          loss = loss_fn(outputs,labels)
          #Backpropagate the loss
          loss.backward()

          #Adjust parameters according to the computed gradients
          optimizer.step()

          train_loss += loss.item() * images.size(0)
          _, prediction = torch.max(outputs.data, 1)
          
          train_acc += torch.sum(prediction == labels.data)
        
      train_acc = train_acc / train_count * 100
      train_loss = train_loss / train_count
      print("Loss: {}, Training accuracy: {}%".format(train_loss, train_acc))
      print(prediction, labels.data)

      #Call the learning rate adjustment function
      adjust_learning_rate(epoch)

      valid_acc = validation()
      print("Validation acc: {}&".format(valid_acc*100))
      if valid_acc > best_acc:
            save_models(epoch)
            best_acc = valid_acc

epoch: 0
Loss: 0.3984510304463249, Training accuracy: 89.0029296875%
tensor([128, 137, 127,  59, 118, 123], device='cuda:0') tensor([128,  61, 101,   2, 118, 123], device='cuda:0')
Validation acc: 53.22581100463867&
Checkpoint saved
epoch: 1
Loss: 0.4088521250818201, Training accuracy: 88.60494232177734%
tensor([ 21,  35,  41, 102, 143,  36], device='cuda:0') tensor([ 89,  35,  41, 102, 143,  36], device='cuda:0')
Validation acc: 54.10557556152344&
Checkpoint saved
epoch: 2
Loss: 0.33191195058753026, Training accuracy: 90.53204345703125%
tensor([127, 117,  78,  36, 130,  59], device='cuda:0') tensor([127, 117,  78,  36, 130,  59], device='cuda:0')
Validation acc: 47.36070251464844&
epoch: 3
Loss: 0.40504979467230906, Training accuracy: 89.40092468261719%
tensor([136,  89,  34, 130, 135,  62], device='cuda:0') tensor([136,  89, 131, 130, 135,  62], device='cuda:0')
Validation acc: 54.10557556152344&
epoch: 4
Loss: 0.29152639159424737, Training accuracy: 91.80980682373047%
tensor([ 91, 1

In [ ]:
torch.save(model.state_dict(), 'model.pt')